### BERTで文書をベクトル化して、そのベクトルを使って検索エンジンを開発する

In [1]:
from transformers import AutoTokenizer, BertModel
import torch


tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese-whole-word-masking")
model = BertModel.from_pretrained("SHONOSUKE/Addtional_Trained_BERT_For_Legal_Domain_v1")


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SHONOSUKE/Addtional_Trained_BERT_For_Legal_Domain_v1 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
text = "私は法律について学んでいます。他にもたくさんの学問を追求していて、その中で法律に興味を持ちました。さらに、法律の中でも特に民法に興味があります。"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)


In [5]:
with torch.no_grad():
    outputs = model(**inputs)

# [CLS]トークンの出力を文書のベクトルとして使用
doc_vector = outputs.last_hidden_state[:, 0, :].squeeze()

print(doc_vector)

tensor([ 2.0742e-01,  8.4026e-01,  3.9895e-01, -6.0953e-01, -1.1792e+00,
        -4.6073e-01,  3.2532e-01,  3.6992e-01,  1.0273e+00,  4.7997e-01,
        -2.1563e-01, -1.5731e-01,  7.6189e-01, -1.6167e-01, -9.3144e-01,
         4.9808e-01, -7.1130e-01,  2.3955e-01,  2.0671e-01, -2.5446e-01,
         1.7583e-01, -1.7249e-01, -8.2031e-01,  9.0697e-01,  5.2804e-01,
         1.6546e-01,  1.0069e-01, -1.2478e+00, -9.8018e-01,  6.9510e-01,
         4.2980e-01, -4.4547e-02, -3.3894e-01,  1.4479e-03,  1.8889e-02,
         4.8537e-01, -6.4466e-01,  1.3762e+00,  1.1029e-02, -1.5044e-01,
        -3.4801e-01, -5.7987e-01, -4.4971e-01,  9.6070e-01, -6.9530e-02,
         2.2931e-02,  2.4871e-01, -4.8105e-01, -1.7094e-01, -5.4150e-02,
         1.4301e-01,  6.1162e-01,  4.5249e-02, -2.8746e-02, -5.0524e-01,
         2.5274e-01, -2.6187e-01, -5.1871e-01, -4.4538e-01, -1.5823e-01,
        -4.2285e-01, -6.3670e-01, -4.8037e-02, -3.2277e-01,  9.9222e-02,
        -5.0372e-01,  5.0011e-01, -1.7863e-01, -4.3

### 実際に

In [1]:
import json
def load_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        return data

clean_dataset = load_file('clean_list.json')

In [4]:
corpus = [value["gist"] for value in clean_dataset.values()]

In [7]:
from transformers import AutoTokenizer, BertModel
import torch


tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese-whole-word-masking")
model = BertModel.from_pretrained("SHONOSUKE/Addtional_Trained_BERT_For_Legal_Domain_v1")


Some weights of BertModel were not initialized from the model checkpoint at SHONOSUKE/Addtional_Trained_BERT_For_Legal_Domain_v1 and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
num_dimention = model.config.hidden_size
print(num_dimention)


768


In [10]:
corpus_vectors = []
num = 1
for text in corpus:
    print(f"{num} / {len(corpus)} is processing")
    num += 1
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    doc_vector = outputs.last_hidden_state[:, 0, :].squeeze()
    corpus_vectors.append(doc_vector)

1 / 30716 is processing
2 / 30716 is processing
3 / 30716 is processing
4 / 30716 is processing
5 / 30716 is processing
6 / 30716 is processing
7 / 30716 is processing
8 / 30716 is processing
9 / 30716 is processing
10 / 30716 is processing
11 / 30716 is processing
12 / 30716 is processing
13 / 30716 is processing
14 / 30716 is processing
15 / 30716 is processing
16 / 30716 is processing
17 / 30716 is processing
18 / 30716 is processing
19 / 30716 is processing
20 / 30716 is processing
21 / 30716 is processing
22 / 30716 is processing
23 / 30716 is processing
24 / 30716 is processing
25 / 30716 is processing
26 / 30716 is processing
27 / 30716 is processing
28 / 30716 is processing
29 / 30716 is processing
30 / 30716 is processing
31 / 30716 is processing
32 / 30716 is processing
33 / 30716 is processing
34 / 30716 is processing
35 / 30716 is processing
36 / 30716 is processing
37 / 30716 is processing
38 / 30716 is processing
39 / 30716 is processing
40 / 30716 is processing
41 / 3071

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from voyager import Index, Space

index = Index(Space.Cosine, num_dimensions=768)
vectors = np.stack(corpus_vectors, axis=0)
_ = index.add_items(vectors)

In [ ]:
index.save("BERT.voy")

In [ ]:
index = Index.load("BERT.voy")

query_documents = [
    "小学校における殺人事件"
]

query_vectors = []
for text in query_documents:
    # 入力をトークナイズし、GPUに移動
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
    doc_vector = outputs.last_hidden_state[:, 0, :].squeeze()

doc_vector_array = doc_vector.cpu().detach().numpy()
neighbors, distances = index.query(doc_vector_array, 5)

neighbors = neighbors.tolist()

for i in range(len(neighbors)):
    print(clean_dataset[list(clean_dataset.keys())[neighbors[i]]]["case_name"])
    print(clean_dataset[list(clean_dataset.keys())[neighbors[i]]]["gist"])
    print(clean_dataset[list(clean_dataset.keys())[neighbors[i]]]["detail_page_link"])
    print("----")